This code creates interpolation data for incidents between the specified
start_date and end_date, and saves it as csv file in the folder:
/fromIncidentToInterpolationData/incidentInterpolationData

In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import json
from sqlalchemy import create_engine

Presets, agreed upon for all diagrams

In [2]:
x_incident = 3000
t_incident = 60
time_recording_before_incident = 60
time_recording_after_incident = 180

mimimal_number_of_cameras = 6


# filter dates between 15th of March 2020 and 21th of March 2020
start_date = '2020-03-22'
end_date = '2020-03-29'

# the filename for the speedflow data 
file_name = f"/Users/vree/Documents/UTwente/Masters UTwente/Data Science/DM Project/Speed-flow/Maart/intensiteit-snelheid-rotterdam-22-29-Maart/intensiteit-snelheid-rotterdam-22-29-Maart.csv"

file_name_for_saving = file_name.split("/")[-1].split(".")[0]


presets for current run

In [3]:
driver='postgresql'
username='dab_ds23241a_223'
dbname=username # it is the same as the username
password='Bh6KFgcWazMMCPvZ'
server='bronto.ewi.utwente.nl'
port='5432'

# Create an engine instance
alchemy_engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')

# Connect to PostgreSQL server
db_connection = alchemy_engine.connect()

# Read the data into a_df
a_df = pd.read_sql(f"select * from \"project\".\"Incidents_With_Cameras\"", db_connection)

# Check if 'before' and 'after' columns are already dictionaries
if a_df['before'].apply(type).eq(dict).all() and a_df['after'].apply(type).eq(dict).all():
    print('Columns are already dictionaries.')
else:
    # Convert the 'before' and 'after' columns from JSON strings back to dictionaries if they are not already
    a_df['before'] = a_df['before'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    a_df['after'] = a_df['after'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# Close the database connection
db_connection.close()
print(a_df.shape)
a_df.head()


(266, 9)


,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,segment,before,after
0,RWS02_0000157188_157188,2020-03-01 08:38:28,vehicle_obstruction,emergencyVehicle,4.366420,51.920952,north,{'RWS01_MONIBAS_0041hrl0706ra': 0.022954588858...,{'GEO0K_K_RWSTI357132': 0.16579609461072695}
1,RWS02_0000157197_157197,2020-03-01 10:00:57,vehicle_obstruction,emergencyVehicle,4.432840,51.928864,west-east,{'RWS01_MONIBAS_0201hrr0284ra': 0.094239550154...,{'RWS01_MONIBAS_0201hrr0285ra': 0.065785586736...
2,RWS02_0000157269_157269,2020-03-01 13:21:58,vehicle_obstruction,emergencyVehicle,4.456025,51.864422,east-west,{'RWS01_MONIBAS_0150vwy0557ra': 0.249505010285...,{'RWS01_MONIBAS_0150vwy0553ra': 0.229508095798...
3,RWS03_816372_1,2020-03-01 15:19:57,vehicle_obstruction,brokenDownVehicle,4.538568,51.909523,south,{'RWS01_MONIBAS_0160vwx0193ra': 0.896763594374...,{'RWS01_MONIBAS_0160vwx0202ra': 0.001663796286...
4,RWS02_0000157393_157393,2020-03-01 23:03:29,vehicle_obstruction,emergencyVehicle,4.439620,51.868599,east-west,{'RWS01_MONIBAS_0150vwy0544ra': 0.166980200322...,{'RWS01_MONIBAS_0150vwy0540ra': 0.210197772623...


Filter between the preset dates

In [4]:
# filter between preset dates: start_date and end_date
a_df = a_df[(a_df['starttijd'] >= start_date) & (a_df['starttijd'] <= end_date)]
print(a_df.shape)

(26, 9)


In [5]:
# returns cameras and their true x for an incident
#  so a list of camera ids and their x values
def create_x_variable_for_incident_cameras(row):
    locations_before = row['before']
    locations_after = row['after']
    
    # create dataframe with distance before, where the row entry is a json with a key
    # for camera id and the value is the distance
    before_cameras_data = []
    
    before_cameras_data = [{'camera_id': key, 'x': x_incident - (value * 1000)} for key, value in locations_before.items()]
    
    before_cameras_x = pd.DataFrame(before_cameras_data)
    
    # create dataframe with distance after, where the row entry is a json with a key
    # for camera id and the value is the distance
    after_cameras_data = []
    
    after_cameras_data = [{'camera_id': key, 'x': x_incident + (value * 1000)} for key, value in locations_after.items()]
        
    after_cameras_x = pd.DataFrame(after_cameras_data)
    df = pd.concat([before_cameras_x, after_cameras_x])
    print(df.head(3))
    return df


 
    
    
    
    
    

In [6]:
# times must be delivered as pd.datetime
def speed_flow_data_for_incident(camera_ids, start_time, end_time):
    # read speed flow data in chunks of 10000 and filter on camera id and time
    # only keep start_meetperiode, id_meetlocatie, gem_snelheid
    
    
    chunksize = 10000
    chunks = []
    for chunk in pd.read_csv(file_name, chunksize=chunksize):
        chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
        chunk = chunk[['start_meetperiode', 'id_meetlocatie', 'gem_snelheid']]
        chunks.append(chunk)
    return pd.concat(chunks)


Creating the incident interpolation data

In [7]:
# incident_row = a_df.iloc[0]

# cameras_x_df = create_x_variable_for_incident_cameras(incident_row)
    
# camera_ids = cameras_x_df['camera_id'].tolist()

# incident_time = pd.to_datetime(incident_row['starttijd'])
# start_time_for_diagram = incident_time - pd.Timedelta(minutes=time_recording_before_incident)
# end_time_for_diagram = incident_time + pd.Timedelta(minutes=time_recording_after_incident)
# # returns df with start_meetperiode, id_meetlocatie, gem_snelheid
# speedflow_data = speed_flow_data_for_incident(camera_ids, start_time_for_diagram, end_time_for_diagram)

# # let the start_meetperiode be the time in minutes since the incident
# speedflow_data['start_meetperiode'] = (pd.to_datetime(speedflow_data['start_meetperiode']) - incident_time).dt.total_seconds() / 60.0

# merged_data = pd.merge(cameras_x_df, speedflow_data, left_on='camera_id', right_on='id_meetlocatie')

# # rename columns: x->x, gem_snelheid->v, start_meetperiode->t
# merged_data = merged_data.rename(columns={'x': 'x', 'gem_snelheid': 'v', 'start_meetperiode': 't'})


In [8]:
# aggregate duplicates where x and t are the same, and take the average of v
def aggregate_data(data):
    return data.groupby(['x', 't']).agg({'v': 'mean'}).reset_index()



In [9]:
# merged_data.head()

In [10]:
# retrieving a list of cameras and their x values
def create_interpolation_data_for_incident(incident_row):
    

    cameras_x_df = create_x_variable_for_incident_cameras(incident_row)
    
        
    camera_ids = cameras_x_df['camera_id'].tolist()

    incident_time = pd.to_datetime(incident_row['starttijd'])
    start_time_for_diagram = incident_time - pd.Timedelta(minutes=time_recording_before_incident)
    end_time_for_diagram = incident_time + pd.Timedelta(minutes=time_recording_after_incident)
    # returns df with start_meetperiode, id_meetlocatie, gem_snelheid
    speedflow_data = speed_flow_data_for_incident(camera_ids, start_time_for_diagram, end_time_for_diagram)

    print("speed flow data for incident was: ", speedflow_data.head())

    # let the start_meetperiode be the time in minutes since the incident
    speedflow_data['start_meetperiode'] = ((pd.to_datetime(speedflow_data['start_meetperiode']) - incident_time).dt.total_seconds() / 60.0)+t_incident

    merged_data = pd.merge(cameras_x_df, speedflow_data, left_on='camera_id', right_on='id_meetlocatie')

    # rename columns: x->x, gem_snelheid->v, start_meetperiode->t
    merged_data = merged_data.rename(columns={'x': 'x', 'gem_snelheid': 'v', 'start_meetperiode': 't'})
    
    return merged_data






    
        
    
    




In [11]:
import os

In [12]:

def check_number_of_cameras(incident_row):
    locations_before = incident_row['before']
    locations_after = incident_row['after']
    
    return len(locations_before) + len(locations_after) > mimimal_number_of_cameras

def create_x_t_v_for_incident_row(incident_row):

    incident_data = create_interpolation_data_for_incident(incident_row)
    

    # only keep the columns x, v, t
    incident_data = incident_data[['x', 'v', 't']]

    # there's multiple measurements within a minute, which lead to duplicates for x, t . 
    incident_data = aggregate_data(incident_data)
    # show number of rows incident data
    print('number of rows incident data: ', len(incident_data))

    incident_data_remove_negatives = incident_data[incident_data['v'] >= 0]
    # show number of rows after removing negatives
    print('number of rows after removing negatives; ', len(incident_data_remove_negatives))

    # Check if the directory exists, if not, create it
    base_path = './incidentInterpolationData/'
    full_path = f"{base_path}/{file_name_for_saving}"
    if not os.path.exists(full_path):
        os.makedirs(full_path)
    # write the data to a csv file in ./incidentInterpolationData/, use the incident id as filename
    incident_data_remove_negatives.to_csv(f'./incidentInterpolationData/{file_name_for_saving}/{incident_row["id"]}.csv', index=False)
    print("created csv for incident: ", incident_row["id"])
    
    return incident_data_remove_negatives


for i in range(0, len(a_df)):
    incident_row = a_df.iloc[i]
    
    if check_number_of_cameras(incident_row):
        create_x_t_v_for_incident_row(incident_row)
    else:
        print(f'incident {incident_row["id"]} has too few cameras')


# let os say that program has been finsihed
import os
os.system('say "your program has finished"')

                     camera_id            x
0  RWS01_MONIBAS_0150vwy0628ra  2953.267858
1  RWS01_MONIBAS_0150vwy0633ra  2475.127253
0  RWS01_MONIBAS_0150vwy0626ra  3138.649010


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not inf

speed flow data for incident was:              start_meetperiode               id_meetlocatie  gem_snelheid
47959221  2020-03-26 23:19:00  RWS01_MONIBAS_0150vwn0579ra          -1.0
47959222  2020-03-26 23:19:00  RWS01_MONIBAS_0150vwn0579ra          -1.0
47959223  2020-03-26 23:19:00  RWS01_MONIBAS_0150vwn0579ra          99.0
47959224  2020-03-26 23:20:00  RWS01_MONIBAS_0150vwn0579ra         111.0
47959225  2020-03-26 23:20:00  RWS01_MONIBAS_0150vwn0579ra          -1.0
number of rows incident data:  3120
number of rows after removing negatives;  2291
created csv for incident:  RWS02_0000170467_170467
                           camera_id            x
0  RWS01_MONICA_10D0040B545B68200005  2957.573929
1  RWS01_MONICA_10D0040B5C2468200005  2814.305573
2        RWS01_MONIBAS_0040vwn0728ra  2672.146902


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not inf

speed flow data for incident was:             start_meetperiode       id_meetlocatie  gem_snelheid
4342770  2020-03-22 15:59:00  GEO0K_K_RWSTI357132           0.0
4342771  2020-03-22 15:59:00  GEO0K_K_RWSTI357132         103.0
4342772  2020-03-22 15:59:00  GEO0K_K_RWSTI357132           0.0
4342773  2020-03-22 15:59:00  GEO0K_K_RWSTI357132           0.0
4342774  2020-03-22 15:59:00  GEO0K_K_RWSTI357132           0.0
number of rows incident data:  4559
number of rows after removing negatives;  4154
created csv for incident:  RWS02_0000168554_168554
                     camera_id            x
0  RWS01_MONIBAS_0201hrr0363ra  2715.314100
1  RWS01_MONIBAS_0201hrr0360ra  2416.477414
2  RWS01_MONIBAS_0201hrr0354ra  1820.302745


...

/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not inf

speed flow data for incident was:              start_meetperiode               id_meetlocatie  gem_snelheid
44351879  2020-03-28 12:31:00  RWS01_MONIBAS_0041hrl0754ra         114.0
44351880  2020-03-28 12:31:00  RWS01_MONIBAS_0041hrl0754ra         101.0
44351881  2020-03-28 12:31:00  RWS01_MONIBAS_0041hrl0754ra         101.0
44351882  2020-03-28 12:31:00  RWS01_MONIBAS_0041hrl0754ra         102.0
44351883  2020-03-28 12:31:00  RWS01_MONIBAS_0041hrl0754ra          96.0
number of rows incident data:  1680
number of rows after removing negatives;  1680
created csv for incident:  RWS02_0000171155_171155
                     camera_id            x
0  RWS01_MONIBAS_0150vwq0572ra  2675.545657
1  RWS01_MONIBAS_0150vwq0577ra  2222.228240
2  RWS01_MONIBAS_0150vwn0579ra  1989.317596


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk = chunk[(chunk['id_meetlocatie'].isin(camera_ids)) & (pd.to_datetime(chunk['start_meetperiode']) >= start_time) & (pd.to_datetime(chunk['start_meetperiode']) <= end_time)]
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_53263/252438380.py:10: UserWarning: Could not inf

speed flow data for incident was:              start_meetperiode               id_meetlocatie  gem_snelheid
47966607  2020-03-28 16:21:00  RWS01_MONIBAS_0150vwn0579ra         104.0
47966608  2020-03-28 16:21:00  RWS01_MONIBAS_0150vwn0579ra         116.0
47966609  2020-03-28 16:21:00  RWS01_MONIBAS_0150vwn0579ra         101.0
47966610  2020-03-28 16:22:00  RWS01_MONIBAS_0150vwn0579ra         102.0
47966611  2020-03-28 16:22:00  RWS01_MONIBAS_0150vwn0579ra          -1.0
number of rows incident data:  4560
number of rows after removing negatives;  4553
created csv for incident:  RWS02_0000171238_171238


0